# 올영 리뷰 데이터 EDA (탐색적 데이터 분석)

이 노트북은 올영 리뷰 데이터에 대한 포괄적인 탐색적 데이터 분석(EDA)을 수행합니다.
- 데이터 구조 및 기본 정보 파악
- 통계 분석 및 분포 확인
- 결측치 및 이상치 탐지
- 데이터 시각화

## 1. 필요한 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings

warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")

print("✓ 라이브러리 임포트 완료")

## 2. 데이터 로드 및 기본 확인

In [ ]:
# JSON 또는 CSV 파일 로드
try:
    # JSON 파일 로드 시도
    df = pd.read_json('data/올영리뷰데이터_utf8.json', encoding='utf-8')
    print("✓ JSON 파일에서 데이터 로드 성공")
except:
    try:
        # CSV 파일 로드 시도
        df = pd.read_csv(
            'data/올영리뷰데이터_utf8.csv',
            encoding='utf-8',
            engine='python',
            on_bad_lines='skip'
        )
        print("✓ CSV 파일에서 데이터 로드 성공")
    except Exception as e:
        print(f"✗ 데이터 로드 실패: {e}")

print(f"\n데이터셋 크기: {df.shape[0]:,}행 × {df.shape[1]} 열")
print(f"\n처음 5개 행:")
print(df.head())

## 3. 데이터 구조 및 정보

In [ ]:
print("=== 데이터 정보 ===\n")
print(df.info())

print("\n=== 컬럼명 및 데이터 타입 ===")
print(df.dtypes)

print("\n=== 데이터 형태 ===")
print(f"행: {df.shape[0]:,}, 열: {df.shape[1]}")

## 4. 결측치 분석

In [ ]:
# 결측치 확인
print("=== 결측치 분석 ===\n")
missing_data = pd.DataFrame({
    '컬럼': df.columns,
    '결측치 수': df.isnull().sum().values,
    '결측치율(%)': (df.isnull().sum().values / len(df) * 100).round(2)
})
print(missing_data)

print(f"\n전체 결측치 비율: {(df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.2f}%")

## 5. 기술통계 (Descriptive Statistics)

In [ ]:
print("=== 수치형 컬럼 기술통계 ===\n")
print(df.describe().round(2))

# 범주형 컬럼 분석
print("\n=== 범주형 컬럼 분석 ===")
for col in df.select_dtypes(include='object').columns:
    print(f"\n{col}:")
    print(df[col].value_counts().head(10))

## 6. 수치형 변수의 분포 시각화

In [ ]:
# 수치형 컬럼 분포 시각화
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if numeric_cols:
    fig, axes = plt.subplots(len(numeric_cols), 2, figsize=(15, 4*len(numeric_cols)))
    
    if len(numeric_cols) == 1:
        axes = axes.reshape(1, -1)
    
    for idx, col in enumerate(numeric_cols):
        # 히스토그램
        axes[idx, 0].hist(df[col].dropna(), bins=30, edgecolor='black', alpha=0.7)
        axes[idx, 0].set_title(f'{col} - Histogram')
        axes[idx, 0].set_xlabel('Value')
        axes[idx, 0].set_ylabel('Frequency')
        
        # 박스플롯
        axes[idx, 1].boxplot(df[col].dropna())
        axes[idx, 1].set_title(f'{col} - Boxplot')
        axes[idx, 1].set_ylabel('Value')
    
    plt.tight_layout()
    plt.show()
    print("✓ 수치형 변수 분포 시각화 완료")

## 7. 범주형 변수의 분포 시각화

In [ ]:
# 범주형 컬럼 분포 시각화
categorical_cols = df.select_dtypes(include='object').columns.tolist()

if categorical_cols:
    # 컬럼당 고유값 개수 확인
    for col in categorical_cols:
        unique_count = df[col].nunique()
        if unique_count <= 10:  # 고유값이 10개 이하인 경우만 시각화
            fig, ax = plt.subplots(figsize=(10, 4))
            df[col].value_counts().plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
            ax.set_title(f'{col} Distribution')
            ax.set_xlabel(col)
            ax.set_ylabel('Count')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
        else:
            print(f"\n{col}: {unique_count}개의 고유값 (상위 10개만 표시)")
            print(df[col].value_counts().head(10))

## 8. 이상치 탐지 (IQR 방법)

In [ ]:
print("=== 이상치 탐지 (IQR 방법) ===\n")

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    
    print(f"{col}:")
    print(f"  Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
    print(f"  Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    print(f"  이상치 개수: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
    print()

## 9. 상관관계 분석

In [ ]:
if len(numeric_cols) > 1:
    # 상관관계 행렬
    correlation_matrix = df[numeric_cols].corr()
    
    print("=== 상관관계 행렬 ===")
    print(correlation_matrix.round(3))
    
    # 상관관계 히트맵 시각화
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
    ax.set_title('Correlation Matrix Heatmap')
    plt.tight_layout()
    plt.show()
else:
    print("수치형 변수가 1개 이하이므로 상관관계 분석을 건너뜁니다.")

## 10. EDA 요약

In [ ]:
print("\n" + "="*60)
print("EDA 분석 요약")
print("="*60 + "\n")

print(f"📊 데이터셋 크기: {df.shape[0]:,}행 × {df.shape[1]} 열")
print(f"📌 수치형 컬럼: {len(numeric_cols)}개")
print(f"📝 범주형 컬럼: {len(categorical_cols)}개")

print(f"\n결측치 현황:")
missing_total = df.isnull().sum().sum()
print(f"  - 총 결측치: {missing_total:,}개")
print(f"  - 결측치 비율: {(missing_total / (df.shape[0] * df.shape[1]) * 100):.2f}%")

print(f"\n데이터 품질:")
if missing_total == 0:
    print("  ✓ 결측치 없음")
else:
    print(f"  ⚠ 결측치 처리 필요")

print("\n✓ EDA 분석 완료!")